# Univariate Sort
1. Load the monthly CRSP file, the Fama-French Factors, and the estimated betas from the tidy_finance_*.sqlite database.

2. Create portfolio sorts based on the lagged beta. Specifically, you compute the breakpoint as the median lag beta each month. Then, you compute the returns of a portfolio that invests only in the stocks with a higher beta than the breakpoint and a portfolio that invests only in the stocks with a lower beta than the breakpoints. The portfolio weights can either be equal or value-weighted.

3. What are the monthly excess returns of both portfolios?

4. Does a portfolio that goes long high beta stocks and short low beta stocks yield an excess return significantly different from zero?

5. Write a general function for portfolio sorts based on a variable number of breakpoints. Then, compute portfolio returns based on lagged beta decile sorts.

6. What is the CAPM alpha of the ten portfolio returns? Is this finding in line with your expectations based on the CAPM implications?

7. Does a high beta minus low beta portfolio yield abnormal excess returns?

In [6]:
import pandas as pd
import sqlite3
from plotnine import *
tidy_finance_python = sqlite3.connect( database="/Users/asbjornfyhn/Desktop/Emp Fin/data/tidy_finance_python.sqlite")
pd.read_sql_query('''SELECT name FROM sqlite_schema WHERE type ='table' AND  name NOT LIKE 'sqlite_%';''', con =tidy_finance_python)

,name
0,trace_enhanced
1,crsp_daily
2,factors_ff3_monthly
3,factors_ff5_monthly
4,factors_ff3_daily
5,industries_ff_monthly
6,factors_q_monthly
7,macro_predictors
8,cpi_monthly
9,fisd


### 1. Part
Load the monthly CRSP file, the Fama-French Factors, and the estimated betas from the tidy_finance_*.sqlite database.

In [16]:
print(pd.read_sql_query('''SELECT * FROM crsp_monthly limit 1 ;''', con =tidy_finance_python).columns)
print(pd.read_sql_query('''SELECT * FROM factors_ff3_monthly limit 1 ;''', con =tidy_finance_python).columns)
print(pd.read_sql_query('''SELECT * FROM beta limit 1 ;''', con =tidy_finance_python).columns)

Index(['permno', 'date', 'month', 'ret', 'shrout', 'altprc', 'exchcd', 'siccd',
       'mktcap', 'mktcap_lag', 'exchange', 'industry', 'ret_excess', 'gvkey'],
      dtype='object')
Index(['month', 'mkt_excess', 'smb', 'hml', 'rf'], dtype='object')
Index(['permno', 'month', 'beta_monthly', 'beta_daily'], dtype='object')


In [17]:
crps_monthly = (pd.read_sql_query('select * from crsp_monthly;',con=tidy_finance_python, parse_dates={"month"}))
ff3_monthly = (pd.read_sql_query('select * from factors_ff3_monthly;',con=tidy_finance_python, parse_dates={"month"}))
beta_monthly = (pd.read_sql_query('select * from beta;',con=tidy_finance_python, parse_dates={"month"}))
